## 1. Importação das bibliotecas

In [1]:
import heapq
import json
import math
import nltk
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import os
import pandas as pd
import spacy
import string
import time

## 2. Pré-Processamento

### 2.1. Definição da função de Pré-processamento

In [2]:
def remove_non_ascii(texto):
    return "".join(c for c in texto if ord(c) < 128)

In [3]:
def remove_special_characters(texto):
    return "".join(c for c in texto if c.isalnum() or c.isspace())

In [4]:
def remove_digits(texto):
    return "".join(c for c in texto if not c.isdigit())

A função **preprocessamento** faz a limpeza no texto removendo todos os caracteres que devem ser desconsiderados no processo de geração de tokens de sentenças e palavras.

In [5]:
# Lista de palavras irrelevantes para a etapa de processamento do texto.
stopwords = nltk.corpus.stopwords.words('english')

# Lista de sinais de pontuação
punctuation = string.punctuation

def preprocessamento(texto):
    texto = texto.lower()
    texto = remove_non_ascii(texto)
    texto = remove_special_characters(texto)
    texto = remove_digits(texto)
    
    #documento = pln(texto)
    
    tokens = []
    
    # Pré-processamento sem a lematização
    for token in nltk.word_tokenize(texto):
        tokens.append(token)
    
    # Pré-processamento com a lematização
    #for token in documento:
        #tokens.append(token.lemma_)

    # Filtrar apenas as palavras que não estiverem na lista de stopwords e nem na lista de pontuações
    tokens = [palavra for palavra in tokens if palavra not in stopwords and palavra not in punctuation]
    
    # Converter a lista de palavras para uma String, não adiciona valores numéricos
    texto = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()])

    return texto

### 2.2. Definição das funções de tokenização

A função **token_sentences** transforma cada sentença do texto original em um token. Cada frase do texto, é considerado um token.

In [6]:
# Transforma cada frase do texto original em um token.
def token_sentences(texto):
    sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto)]
    
    return sentencas_originais

A função **preprocessing_sentences** recebe uma lista de sentenças com o texto original e executa a função de preprocessamento, eliminando caracteres especiais, sinais de pontuação, dígitos, stop words, etc. Essa função retorna uma lista de sentenças formatadas (pré-processadas)

No algoritmo da similaridade do cosseno, a sentença formatada não pode ser vazia. Nesse caso, acrescenta-se um caractere especial que não influenciará no resultado do algoritmo.

In [7]:
# Pré-processamento das sentenças originais
def preprocessing_sentences(sentencas_originais):
    sentencas_formatadas = []
    for sentenca_original in sentencas_originais:
        sentenca_formatada = preprocessamento(sentenca_original)
        if sentenca_formatada:
            sentencas_formatadas.append(sentenca_formatada)
        else:
            sentencas_formatadas.append('#')
    
    return sentencas_formatadas

## 3. Algoritmo de Similaridade do Cosseno

### 3.1. Definição da função sentences_similarity

In [8]:
def sentences_similarity(sentenca1, sentenca2):
    
    # Transforma cada sentença em uma lista de palavras
    palavras1 = [palavra for palavra in nltk.word_tokenize(sentenca1)]
    palavras2 = [palavra for palavra in nltk.word_tokenize(sentenca2)]

    # Concatena todas as palavras das duas sentenças, desconsiderando as repetições
    todas_palavras = list(set(palavras1 + palavras2))

    # Criação de vetores com as quantidades de palavras das duas sentenças
    # Os vetores são iniciados em todas as posições com o valor zero
    vetor1 = [0] * len(todas_palavras)
    vetor2 = [0] * len(todas_palavras)

    # Quando encontrar uma palavra que esteja na lista, fará um incremento na posição que a palavra ocupa no vetor
    # Dessa forma, teremos o número de vezes que cada palavra está presente em cada sentença
    for palavra in palavras1:
        vetor1[todas_palavras.index(palavra)] += 1
    
    for palavra in palavras2:
        vetor2[todas_palavras.index(palavra)] += 1
    
    # Cálculo da similaridade entre os dois vetores
    return 1 - cosine_distance(vetor1, vetor2)

### 3.2. Definição da função similarity_matrix

In [9]:
def similarity_matrix(sentencas):
    
    # Inicialização da matriz com todos os valores iguais a zero
    matriz_similaridade = np.zeros((len(sentencas), len(sentencas)))
    
    for i in range(len(sentencas)):
        for j in range(len(sentencas)):
            if i == j:
                continue
            matriz_similaridade[i][j] = round(sentences_similarity(sentencas[i], sentencas[j]), 2)

    return matriz_similaridade

### 3.3. Definição da função summarize

In [10]:
def summarize(sentencas_originais, sentencas_formatadas, quantidade_sentencas):
    
    matriz_similaridade = similarity_matrix(sentencas_formatadas)
    
    # Utilizar a matriz para realizar o cálculo da nota de cada sentença
    # Transformar a matriz em um grafo
    grafo_similaridade = nx.from_numpy_array(matriz_similaridade)
    
    # Nós do grafo
    #print(grafo_similaridade.nodes)
    
    # Arestas do grafo
    #print(grafo_similaridade.edges)
    
    # Calcula o ranking dos nós do grafo baseado na estrutura dos links
    # A sentença que tiver mais ligações é considerada mais importante
    # Pagerank: algoritmo desenvolvido pelo Google para trabalhar com o ranking de páginas web
    notas = nx.pagerank(grafo_similaridade)
    
    # Ordenando as sentenças por notas
    notas_ordenadas = sorted(((notas[i], nota) for i, nota in enumerate(sentencas_originais)), reverse=True)
    
    # Lista com as melhores sentenças
    melhores_sentencas = []
    for i in range(quantidade_sentencas):
        melhores_sentencas.append(notas_ordenadas[i][1])
  
    return sentencas_originais, melhores_sentencas, notas_ordenadas

### 3.4. Definição da função view_summary

In [11]:
def view_summary(lista_sentencas, melhores_sentencas):
    texto = ''
    
    for sentenca in lista_sentencas:
        if sentenca in melhores_sentencas:
            texto = texto + ' ' + sentenca
    return texto

### 3.5. Execução do Algoritmo para todas as notícias

Esse trecho de código corresponde à execução completa do **Algoritmo de Similaridade do Cosseno** para um lote de notícias. Os seguintes passos são executados:



In [12]:
# Leitura do arquivo JSON com a base de dados de notícias
dt = pd.read_json("news.json")
display(dt)

,titulo,conteudo,sumario
0,Daman & Diu revokes mandatory Rakshabandhan in...,The Daman and Diu administration on Wednesday ...,The Administration of Union Territory Daman an...
1,Malaika slams user who trolled her for 'divorc...,"From her special numbers to TVappearances, Bol...",Malaika Arora slammed an Instagram user who tr...
2,'Virgin' now corrected to 'Unmarried' in IGIMS...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Aaj aapne pakad liya: LeT man Dujana before be...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotel staff to get training to spot signs of s...,Hotels in Mumbai and other Indian cities are t...,Hotels in Maharashtra will train their staff t...
...,...,...,...
3960,Rasna seeking ?250 cr revenue from snack categ...,"Mumbai, Feb 23 (PTI) Fruit juice concentrate m...",Fruit juice concentrate maker Rasna is eyeing ...
3961,Sachin attends Rajya Sabha after questions on ...,Former cricketer Sachin Tendulkar was spotted ...,Former Indian cricketer Sachin Tendulkar atten...
3962,Shouldn't rob their childhood: Aamir on kids r...,"Aamir Khan, whose last film Dangal told the st...","Aamir Khan, while talking about reality shows ..."
3963,"Asha Bhosle gets ?53,000 power bill for unused...",Maharahstra Power Minister Chandrashekhar Bawa...,The Maharashtra government has initiated an in...


In [13]:
# Lista para armazenar o resultado do processamento do Algoritmo de Similaridade do Cosseno para todas as notícias
algoritmo_cosseno = []
processamento_cosseno = []

inicio = time.time()
processamento_cosseno.append({'Begin': inicio})
print('Begin:', inicio)

for i in range(len(dt)):
    titulo = dt.iloc[i, 0]
    conteudo = dt.iloc[i, 1]
    sumario = dt.iloc[i, 2]
    
    # Geração de tokens (sentenças) com os textos originais
    sentencas_originais = token_sentences(conteudo)
    
    # Executar o pre-processamento das sentenças para eliminar os caracteres e palavras irrelevantes
    sentencas_formatadas = preprocessing_sentences(sentencas_originais)
    
    # Quantidade de sentenças que irão compor o sumário: 20% do total de sentenças
    quantidade_sentencas = math.ceil(len(sentencas_originais)*0.2)
    
    # Geração das melhores sentenças que irão compor o sumário do texto
    sentencas_originais, melhores_sentencas, notas_sentencas = summarize(sentencas_originais, sentencas_formatadas, quantidade_sentencas)
    
    # Geração do sumário somente com as frases correspondentes às melhores sentenças do texto
    sumario_cosine = view_summary(sentencas_originais, melhores_sentencas)
    
    algoritmo_cosseno.append({'index': i, 'titulo': titulo, 'conteudo': conteudo, 'sumario': sumario, 'sumario_cosine': sumario_cosine})
    if i % 100 == 0:
        print(i)

fim = time.time()
print('Fim da execução:', fim)
processamento_cosseno.append({'End': fim})

duracao = fim - inicio
print('Duração:', duracao)
processamento_cosseno.append({'Duration': duracao})

Begin: 1635870784.8832512
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
Fim da execução: 1635871784.0270329
Duração: 999.1437816619873


In [14]:
# Será criado um arquivo algoritmo_cosseno.json no mesmo diretório de execução do programa
arquivo_gravar = os.path.join('algoritmo_cosseno.json')
arquivo = open(arquivo_gravar, 'w+')
arquivo.write(json.dumps(algoritmo_cosseno, indent=1))
arquivo.close()

In [15]:
# Será criado um arquivo processamento_cosseno.json no mesmo diretório de execução do programa
arquivo_gravar = os.path.join('processamento_cosseno.json')
arquivo = open(arquivo_gravar, 'w+')
arquivo.write(json.dumps(processamento_cosseno, indent=1))
arquivo.close()